In [1]:
import os

from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.schema import Column
from sqlalchemy.types import Integer, String

In [2]:
load_dotenv()
dbname = os.environ['POSTGRES_DB']
username = os.environ['POSTGRES_USER']
password = os.environ['POSTGRES_PASSWORD']
host = 'postgres'
port = '5432'

In [3]:
conn = psycopg2.connect(dbname=dbname, user=username, password=password, host=host, port=port)
with conn:
    with conn.cursor() as curs:
        curs.execute('SELECT VERSION()')
        print(curs.fetchone())
conn.close()

('PostgreSQL 14.4 (Debian 14.4-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


In [4]:
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}')
Base = declarative_base()


class User(Base):
    __tablename__ = 'users'
    user_id = Column(Integer, primary_key=True)
    username = Column(String(255))
    email = Column(String(255))

    def full_name(self):
        return f'{self.username} ({self.email})'
    
    
Base.metadata.create_all(engine)

Session = sessionmaker(engine)

In [5]:
session = Session()

user_1 = User(username='user1', email='user1@example.com')
user_2 = User(username='user2', email='user2@example.com')

try:
    session.add(user_1)
    session.add(user_2)
    session.commit()
finally:
    session.close()

In [6]:
session = Session()

try:
    users = session.query(User).all()
    print([user.full_name() for user in users])
    user = session.query(User).first()
    print(user.full_name())
finally:
    session.close()

['user1 (user1@example.com)', 'user2 (user2@example.com)']
user1 (user1@example.com)


In [7]:
session = Session()

try:
    user_1 = session.query(User).get(1)
    user_1.username = 'udpated_user'
    session.commit()
    print(user_1.full_name())
finally:
    session.close()

udpated_user (user1@example.com)


In [8]:
session = Session()

try:
    user_1 = session.query(User).get(1)
    user_2 = session.query(User).get(2)
    session.delete(user_1)
    session.delete(user_2)
    session.commit()
finally:
    session.close()

In [9]:
user_3 = User(username='user3', email='user3@example.com')
user_4 = User(username='user4', email='user4@example.com')

with Session() as session:
    session.add(user_3)
    session.add(user_4)
    session.commit()
    users = session.query(User).all()
    print([user.full_name() for user in users])
    user = session.query(User).first()
    print(user.full_name())
    user_3 = session.query(User).get(3)
    user_3.username = 'udpated_user'
    session.commit()
    user_3 = session.query(User).get(3)
    print(user_3.full_name())
    user_3 = session.query(User).get(3)
    user_4 = session.query(User).get(4)
    session.delete(user_3)
    session.delete(user_4)
    session.commit()

['user3 (user3@example.com)', 'user4 (user4@example.com)']
user3 (user3@example.com)
udpated_user (user3@example.com)


In [10]:
Base.metadata.drop_all(engine)